# Reconnaissance de signatures

### Import des librairies

In [1]:
import pygame
from pygame.locals import *;
from math import sin, cos, atan2, sqrt, pi
from xml.dom import minidom
import pandas as pd
import pprint

- pygame : Librairie de jeux vidéo, président ici pour la gesture
- pygame.locals import * : 
- math import sin, cos, atan2, sqrt, pi : 
- xml.dom import minidom : 
- pandas as pd : 
- numpy as np : 
- pprint :

### Variables super globales à renseigner (1ère lettre en majuscule)

In [2]:
Nom = "GALBY"

Prenom = "William"

#Taille des signatures, ex : 250 si fenetre de 500 généralement.
Size = 250

#Précision de la véracité en pourcent,
#sachant que plus on s'approche de 0% plus la signature une copie conforme.
#Si vous choisissez 30%/200% par exemple, 
#cela signifit que la moyenne de tous les pourcentages de véracité du dataset est inférieur à 20%.
Precision = 70

### 1ére etape : Création d'un jeu de donnée saisie directement de l'utilisateur

In [3]:
pygame.init();
screen = pygame.display.set_mode((500,500));
pygame.display.flip();
screen.fill((0,0,0))
running = True
recording = False
points = []
signatures = []
while running:
    events = pygame.event.get();
    for e in events:
        if e.type==MOUSEBUTTONDOWN:
            screen.fill((0,0,0))
            points = []
            recording = True
        if e.type==MOUSEBUTTONUP:
            recording = False
            if len(points) > 10:
                signatures.append(points)
        if e.type==MOUSEMOTION:
            if recording:
                mousepos= pygame.mouse.get_pos()
                points.append(list(mousepos))
                pygame.draw.circle(screen,(255,255,255), mousepos, 3);
        if e.type==QUIT:
            running = 0
    pygame.time.wait(2);
    pygame.display.flip();
pp = pprint.PrettyPrinter(indent=2)
print "Jeu de données brut :"
pp.pprint(signatures)

Jeu de données brut :
[ [ [169, 187],
    [172, 212],
    [178, 234],
    [184, 255],
    [192, 268],
    [195, 274],
    [197, 277],
    [200, 278],
    [204, 278],
    [209, 270],
    [215, 252],
    [218, 239],
    [224, 228],
    [225, 222],
    [227, 218],
    [227, 217],
    [227, 216],
    [228, 215],
    [229, 215],
    [230, 215],
    [232, 217],
    [235, 222],
    [241, 228],
    [246, 234],
    [250, 238],
    [254, 243],
    [255, 244],
    [258, 245],
    [259, 245],
    [262, 245],
    [267, 235],
    [271, 220],
    [275, 204],
    [278, 194],
    [278, 184],
    [278, 174],
    [278, 168],
    [278, 162],
    [278, 161]],
  [ [177, 198],
    [177, 199],
    [177, 201],
    [177, 208],
    [179, 226],
    [185, 241],
    [189, 254],
    [192, 263],
    [195, 267],
    [198, 269],
    [200, 271],
    [203, 272],
    [207, 270],
    [213, 260],
    [221, 245],
    [225, 234],
    [228, 228],
    [228, 225],
    [229, 223],
    [230, 223],
    [231, 223],
    [234, 224],
 

### Jeu de données de points représentant une signature

In [4]:
class Template:
    def __init__(self, name, points):
        self.points = points
        self.name = name

    def prepare(self):
        self.points = resample(self.points, 64)
        self.points = rotate_to_zero(self.points)
        self.points = scale_to_square(self.points, 250)
        self.points = translate_to_origin(self.points)

templates = []
i = 0
for s in signatures:
    templates.append(Template("numéro "+`i`+" de "+Nom+" "+Prenom, s))
    i = i+1

### Fonctions servant à la reconnaisance

In [5]:
def average(xs): return sum(xs) / len(xs)

def resample(points, n):#{{{
    I = pathlength(points) / float(n-1)
    D = 0
    newPoints = [points[0]]
    i = 1
    while i<len(points):
        p_i = points[i]
        d = distance(points[i-1], p_i)
        if (D + d) >= I:
            qx = points[i-1][0] + ((I-D) / d) * (p_i[0] - points[i-1][0])
            qy = points[i-1][1] + ((I-D) / d) * (p_i[1] - points[i-1][1])
            newPoints.append([qx,qy])
            points.insert(i, [qx,qy])
            D = 0
        else: D = D + d
        i+=1
    return newPoints
#}}}

def pathlength(points):#{{{
    d = 0
    for i,p_i in enumerate(points[:len(points)-1]):
        d += distance(p_i, points[i+1])
    return d
#}}}

def distance(p1, p2): return float(sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2))

def centroid(points): return float(average([float(i[0]) for i in points])), float(average([float(i[1]) for i in points]))

def rotate_to_zero(points):#{{{
    cx, cy = centroid(points)
    theta = atan2(cy - points[0][1], cx - points[0][0])
    newPoints = rotate_by(points, -theta)
    return newPoints
#}}}

def rotate_by(points, theta):#{{{
    cx, cy = centroid(points)
    newpoints = []
    cos_p, sin_p = cos(theta), sin(theta)
    for p in points:
        qx = (p[0] - cx) * cos_p - (p[1] - cy) * sin_p + cx
        qy = (p[0] - cx) * sin_p + (p[1] - cy) * cos_p + cy
        newpoints.append([qx,qy])
    return newpoints
#}}}

def bounding_box(points): #{{{
    minx, maxx = min((p[0] for p in points)), max((p[0] for p in points))
    miny, maxy = min((p[1] for p in points)), max((p[1] for p in points))
    return minx, miny, maxx-minx, maxy - miny 
#}}}

def scale_to_square(points, Size):#{{{
    min_x, min_y, w, h = bounding_box(points)
    newPoints = []
    for p in points:
        qx = p[0] * (float(Size) / w )
        qy = p[1] * (float(Size) / h )
        newPoints.append([qx,qy])
    return newPoints
#}}}

def translate_to_origin(points):#{{{
    cx, cy = centroid(points)
    newpoints = []
    for p in points:
        qx, qy = p[0] - cx , p[1] - cy
        newpoints.append([qx,qy])
    return newpoints
#}}}

psi = .5 * (-1 + sqrt(5.))
def distance_at_best_angle(points, T, ta, tb, td):#{{{
    x1 = psi * ta + (1-psi)*tb
    f1 = distance_at_angle(points, T, x1)
    x2 = (1-psi)*ta + psi*tb
    f2 = distance_at_angle(points, T, x2)
    while abs(tb - ta) > td:
        if f1 < f2:
            tb,x2,f2 = x2, x1, f1
            x1 = psi*ta + (1-psi)*tb
            f1 = distance_at_angle(points, T, x1)
        else:
            ta,x1,f1 = x1, x2, f2
            x2 = (1 - psi)*ta + psi*tb
            f2 = distance_at_angle(points, T, x2)
    return min(f1, f2)
#}}}

def distance_at_angle(points, T, theta):#{{{
    newpoints = rotate_by(points, theta)
    d = pathdistance(newpoints, T)
    return d
#}}}

def pathdistance(a,b):#{{{
    d = 0
    for ai, bi in zip(a,b):
        d += distance(ai, bi)
    return d / len(a)
#}}}

def classify(candidate_points,templates, screen, debug=False):#{{{
    np = resample(candidate_points,64)
    np = rotate_to_zero(np)
    if debug: plot(np, (255,0,0), screen)
    np = scale_to_square(np, Size)
    if debug: plot(np, (255,0,255), screen)
    np = translate_to_origin(np)
    if debug: plot(np, (0,255,255), screen)
    template, score = recognize(np, templates)
    return template
#}}}
    
def plot(points, color, screen, Size=1):#{{{
    for p in points:
        pygame.draw.circle(screen,color, p, 1);
        pygame.display.flip()
#}}}

def load_template(xmlfile):#{{{   Returns a set of points loaded from xml file
    points = []
    p_xml = minidom.parse(open(xmlfile,'r'))
    for tag in p_xml.getElementsByTagName("Point"):
        points.append([int(tag.attributes['X'].value), int(tag.attributes['X'].value)])
    return points
#}}}

def sort_colonne_decroissant(x, y):#{{{
    for i in range(len(x)-1, -1, -1):
        if x[i] < y[i]:
            return -1
        elif x[i] > y[i]:
            return 1
    return 0

### Fonction de reconnaisance

In [6]:
def recognize(points, templates):#{{{
    b = float("inf")
    #theta = pi / 4 #45 degrees
    theta = 45.
    #t_d = 2 * (pi / 180)
    t_d = 2.
    Tp = None
    num = 0
    dataOutput = []
    for i,T in enumerate(templates):
        listDim1 = []
        Tpoints = T.points
        d = distance_at_best_angle(points, Tpoints, -theta, theta, t_d)
        #ajoute du Nom
        listDim1.append(T.name)
        #ajoute du score
        listDim1.append(d)
        #ajoute dans la ligne dans le tableau de sortie
        dataOutput.append(listDim1)
        if d < b:
            b = d
            Tp = T
    #création du dataFrame de sortie
    columns = ['Infos signature', 'Score en %']
    index = ['Signature']*len(dataOutput)
    dataOutput.sort(sort_colonne_decroissant)
    df = pd.DataFrame(dataOutput, index=index, columns=columns)
    print ">>> Résultat de la reconaissance pour chaque signature du dataset :"
    print df
    print "\n"
    print ">>> La moyenne de chaque véracité du dataset est de :", df["Score en %"].mean()
    print "\n"
    if df["Score en %"].mean() > Precision :
        print ">>> Cette signature n'est pas celle de " + Nom + " " + Prenom + "\n"
    else :
        print ">>> Cette signature est bien celle de " + Nom + " " + Prenom + "\n"
    return Tp, 1-(b/(0.5 * sqrt(Size*Size * 2)))
#}}}

### Programme principal

In [7]:
def main():
    for t in templates: t.prepare()
    pygame.init();
    screen = pygame.display.set_mode((500,500));
    pygame.display.flip();
    screen.fill((0,0,0))
    running = True
    recording = False
    points = []
    while running:
        events = pygame.event.get();
        for e in events:
            if e.type==MOUSEBUTTONDOWN:
                screen.fill((0,0,0))
                points = []
                recording = True
            if e.type==MOUSEBUTTONUP:
                recording = False
                if len(points) > 10:
                    template_match = classify(points, templates,screen)
                    if template_match : print ">>> La signature du dataset qui s'en approche le plus est la", template_match.name
            if e.type==MOUSEMOTION:
                if recording:
                    mousepos= pygame.mouse.get_pos()
                    points.append(list(mousepos))
                    pygame.draw.circle(screen,(255,255,255), mousepos, 3);
            if e.type==QUIT:
                running = 0
        pygame.time.wait(2);
        pygame.display.flip();
if __name__=="__main__": main()

>>> Résultat de la reconaissance pour chaque signature du dataset :
                     Infos signature  Score en %
Signature  numéro 1 de GALBY William   14.200817
Signature  numéro 2 de GALBY William   28.116388
Signature  numéro 0 de GALBY William   28.127405
Signature  numéro 3 de GALBY William   31.941700


>>> La moyenne de chaque véracité du dataset est de : 25.5965776401


>>> Cette signature est bien celle de GALBY William

>>> La signature du dataset qui s'en approche le plus est la numéro 1 de GALBY William


### Création du tableau de sortie (comme évoqué dans ma proposition)

- Nom de l'étudiants STRING 40
- Prénom de l'étudiants STRING 50
- Précision FLOAT de O à 1
- Score de la véracité de la signature FLOAT de O à 1

_William GALBY, dans le cadre du cours de Machine Leaning de Jeff Abrahamson - Basé sur l'algo "$1" de Jacob O. Wobbrock, Andrew D. Wilson, Yang Li_